In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

### Problem 1

In [2]:
file_name = '2024_HW4.1Data.xlsx'

rts_fund = pd.read_excel(file_name, 'Funds').set_index('Index')
rts_bm = pd.read_excel(file_name, 'Benchmarks').set_index('Index')

In [3]:
res = pd.DataFrame(index=rts_bm.columns, columns=rts_fund.columns)
res.loc['Alpha', :] = np.NaN

for fund in rts_fund.columns:
    alpha = cp.Variable(shape=(1,), name='alpha')
    beta = cp.Variable(shape=rts_bm.shape[1], name='beta')
    epsilon = cp.Variable(shape=rts_bm.shape[0], name='epsilon')
    
    constraints = list()
    constraints.append(
        rts_bm.values @ beta + alpha + epsilon == rts_fund[fund].values
    )
    constraints.append(
        cp.sum(beta) == 1
    )
    constraints.append(
        beta >= 0
    )
    
    obj = cp.Minimize(cp.sum(epsilon**2))
    prob = cp.Problem(objective=obj, constraints=constraints)
    prob.solve()
    
    res.loc[rts_bm.columns, fund] = np.round(beta.value, 8)
    res.loc['Alpha', fund] = np.round(alpha.value, 8)
res

,Fund A,Fund B,Fund C,Fund D
XLB,-0.0,0.137008,0.0,0.0
XLY,-0.0,0.130248,0.3597,0.056174
XLP,-0.0,-0.0,0.0,0.05733
XLE,1.0,0.199424,0.0,0.112449
XLF,-0.0,0.367483,0.0,0.621596
XLV,-0.0,-0.0,0.0,-0.0
XLI,-0.0,0.165838,-0.0,0.037589
XLK,-0.0,0.0,0.6403,-0.0
XLU,-0.0,-0.0,-0.0,0.114862
Alpha,-0.00089616,-8.501e-05,0.00018476,-0.000557


### Problem 2

In [4]:
c = cp.Variable(shape=(1,), name='chairs', nonneg=True)
t = cp.Variable(shape=(1,), name='tables', nonneg=True)
constraints = list()
constraints.append(
    5*c + 40*t <= 4000
)
constraints.append(
    4*c + 10*t <= 1800
)
constraints.append(
    c - 17 >= 4*(t-10)
)

obj = cp.Maximize(2*cp.log(c) + 148*c + 20*cp.log(t) + 900*t)
prob = cp.Problem(objective=obj, constraints=constraints)
prob.solve(solver=cp.CLARABEL)
print(f'Chairs: {np.round(c.value, 2)}, Tables: {np.round(t.value, 2)}')

Chairs: [290.91], Tables: [63.64]
